Link to GitHub repository [here](https://github.com/jesp9435/ComSocSci)

Group member contributions: Both group members contributed equally to the parts of the assignment. We have worked collaboratively on all parts. 

# Part 1: Properties of the real-world network of Computational Social Scientists

In [15]:
# Relevant imports:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import networkx as nx
import netwulf as nw
import numpy as np
import pickle

In [39]:
# These numbers were found in Assignment 1
amount_of_nodes = 14196
amount_of_edges = 54304

# We use equation 3.2 from the book and solve for p:
probability = amount_of_edges/((amount_of_nodes*(amount_of_nodes-1))/2)
print(probability)

# The average degree can be found from equation 3.3:
k = probability*(amount_of_nodes-1.0)
print(k)

# Generating a Random Network with node count mirroring Computational Social Scientists network
# using our calculated probability:
RG = nx.gnp_random_graph(amount_of_nodes, probability, seed=1000, directed=False)
nw.interactive.visualize(RG)

# Computing distribution of degree for the random network:
degrees_random = [degree for node, degree in RG.degree()]
hist, bins = np.histogram(degrees_random, bins=20, density=True)

# Plot the degree distribution
plt.figure(figsize=(6, 4))
plt.hist(degrees_random, bins=bins, density=True, alpha=0.7, edgecolor = "black")
plt.xlabel('Degree')
plt.xlim(0,20)
plt.ylabel('Frequency')
plt.title('Degree Distribution of Random Network')
plt.grid(True)
plt.show()

# Accessing the network from Assignment 1:
with open("my_network.pickle", "rb") as f:
    G = pickle.load(f)

# Computing distribution of degree for the Social network:
degrees_social = [degree for node, degree in G.degree()]
hist, bins = np.histogram(degrees_social, bins=200, density=False)

# Plot the degree distribution
plt.figure(figsize=(6, 4))
plt.hist(degrees_social, bins=bins, density=True, alpha=0.7, edgecolor = "black")
plt.xlabel('Degree')
plt.xlim(0,20)
plt.ylabel('Frequency')
plt.title('Degree Distribution of Social Network')
plt.grid(True)
plt.show()




0.0005389648330011947
7.650605804451958


# Part 2: Network Analysis in Computational Social Science

# Part 3: Words that characterize Computational Social Science communities

##### Exercise 1: TF-IDF and the Computational Social Science communities

* TF-IDF is a statistical measure used in natural language processing and information retrieval to evaluate the importance of a word in a document within a larger collection of documents. The main purpose of TF-IDF is to highlight words that are significant to a specific document while downplaying words that are common across many documents.
* TF stands for Term Frequency and
* IDF stands for Inverse Document Frequency